In [1]:
import h2o
import pandas as pd

In [2]:
from h2o.automl import H2OAutoML

h2o.init(max_mem_size='30G')

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.11" 2021-04-20; OpenJDK Runtime Environment (build 11.0.11+9-Ubuntu-0ubuntu2.18.04); OpenJDK 64-Bit Server VM (build 11.0.11+9-Ubuntu-0ubuntu2.18.04, mixed mode, sharing)
  Starting server from /home/tunguz/.local/lib/python3.6/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp7vv18fua
  JVM stdout: /tmp/tmp7vv18fua/h2o_tunguz_started_from_python.out
  JVM stderr: /tmp/tmp7vv18fua/h2o_tunguz_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,America/Los_Angeles
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.3
H2O_cluster_version_age:,1 month and 15 days
H2O_cluster_name:,H2O_from_python_tunguz_ka5o0h
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,30 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"accepting new members, healthy"


In [3]:
train = h2o.import_file("../input/train.csv")
test = h2o.import_file("../input/test.csv")

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [4]:
train.head()

date_time,deg_C,relative_humidity,absolute_humidity,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,target_carbon_monoxide,target_benzene,target_nitrogen_oxides
2010-03-10 18:00:00,13.1,46,0.7578,1387.2,1087.8,1056,1742.8,1293.4,2.5,12,167.7
2010-03-10 19:00:00,13.2,45.3,0.7255,1279.1,888.2,1197.5,1449.9,1010.9,2.1,9.9,98.9
2010-03-10 20:00:00,12.6,56.2,0.7502,1331.9,929.6,1060.2,1586.1,1117,2.2,9.2,127.1
2010-03-10 21:00:00,11,62.4,0.7867,1321,929,1102.9,1536.5,1263.2,2.2,9.7,177.2
2010-03-10 22:00:00,11.9,59,0.7888,1272,852.7,1180.9,1415.5,1132.2,1.5,6.4,121.8
2010-03-10 23:00:00,11.2,56.8,0.7848,1220.9,697.5,1417.2,1462.6,949,1.2,4.4,88.1
2010-03-11 00:00:00,10.7,55.7,0.7603,1244.2,669.3,1491.2,1413,769.6,1.2,3.7,59.5
2010-03-11 01:00:00,10.3,57,0.7702,1181.4,631.7,1511.1,1359.7,715.4,1,3.4,63.9
2010-03-11 02:00:00,10.1,62.7,0.7648,1159.6,602.9,1610.6,1212.2,657.2,0.9,2.2,46.4
2010-03-11 03:00:00,10.5,59.6,0.7517,1030.2,521.7,1790.2,1148.6,491,0.6,1.6,43


In [5]:
test.head()

date_time,deg_C,relative_humidity,absolute_humidity,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5
2011-01-01 00:00:00,8,41.3,0.4375,1108.8,745.7,797.1,880,1273.1
2011-01-01 01:00:00,5.1,51.7,0.4564,1249.5,864.9,687.9,972.8,1714
2011-01-01 02:00:00,5.8,51.5,0.4689,1102.6,878,693.7,941.9,1300.8
2011-01-01 03:00:00,5,52.3,0.4693,1139.7,916.2,725.6,1011,1283
2011-01-01 04:00:00,4.5,57.5,0.465,1022.4,838.5,871.5,967,1142.3
2011-01-01 05:00:00,4.5,53.7,0.4759,1004,745.5,914.2,989.1,973.8
2011-01-01 06:00:00,3.3,54.8,0.4636,940.9,738.2,816,896.8,1049.4
2011-01-01 07:00:00,3.2,60.7,0.4667,954.5,713.9,834.7,935.6,956.3
2011-01-01 08:00:00,2.5,65.7,0.4721,969.9,679.1,943.8,959.3,892
2011-01-01 09:00:00,3.9,57.8,0.4807,976.6,655.5,996,906,817.5


In [6]:
x = test.columns[1:]
y = 'target_carbon_monoxide'

In [7]:
# Run AutoML for 20 base models (limited to 1 hour max runtime by default)
aml = H2OAutoML(max_models=180, seed=47, max_runtime_secs=600)
aml.train(x=x, y=y, training_frame=train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [8]:
# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

model_id,mean_residual_deviance,rmse,mse,mae,rmsle
StackedEnsemble_AllModels_AutoML_20210705_162650,0.201967,0.449408,0.201967,0.277844,0.124858
StackedEnsemble_BestOfFamily_AutoML_20210705_162650,0.203786,0.451427,0.203786,0.279025,0.125744
GBM_grid__1_AutoML_20210705_162650_model_13,0.211356,0.459734,0.211356,0.285869,0.128429
GBM_grid__1_AutoML_20210705_162650_model_16,0.212202,0.460653,0.212202,0.286114,0.128458
GBM_grid__1_AutoML_20210705_162650_model_18,0.212398,0.460867,0.212398,0.286016,0.129593
GBM_2_AutoML_20210705_162650,0.212918,0.461431,0.212918,0.287371,0.129669
DRF_1_AutoML_20210705_162650,0.213108,0.461636,0.213108,0.286157,0.129408
GBM_4_AutoML_20210705_162650,0.213266,0.461807,0.213266,0.28594,0.129129
GBM_grid__1_AutoML_20210705_162650_model_20,0.213519,0.462081,0.213519,0.286788,0.12907
XRT_1_AutoML_20210705_162650,0.213767,0.46235,0.213767,0.284883,0.12932


In [9]:
# The leader model is stored here
aml.leader

Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_AllModels_AutoML_20210705_162650

No model summary for this model

ModelMetricsRegressionGLM: stackedensemble
** Reported on train data. **

MSE: 0.04001734233154152
RMSE: 0.2000433511305525
MAE: 0.13822104124482085
RMSLE: 0.06418887763072893
R^2: 0.9808879752142067
Mean Residual Deviance: 0.04001734233154152
Null degrees of freedom: 7110
Residual degrees of freedom: 7093
Null deviance: 14889.229399237676
Residual deviance: 284.56332131959175
AIC: -2668.199806555267

ModelMetricsRegressionGLM: stackedensemble
** Reported on cross-validation data. **

MSE: 0.2019674197867671
RMSE: 0.4494078546117848
MAE: 0.27784410832195733
RMSLE: 0.12485844500993311
R^2: 0.9035416619897578
Mean Residual Deviance: 0.2019674197867671
Null degrees of freedom: 7110
Residual degrees of freedom: 7092
Null deviance: 14889.735004302205
Residual deviance: 1436.190322103701
AIC: 8845.040613833846


In [10]:
# If you need to generate predictions on a test set, you can make
# predictions directly on the `"H2OAutoML"` object, or on the leader
# model object directly

preds = aml.predict(test)

stackedensemble prediction progress: |████████████████████████████████████| 100%


In [11]:
sample_submission = pd.read_csv('../input/sample_submission.csv')
sample_submission[y] = preds.as_data_frame().values.flatten()
sample_submission.to_csv(f'../submissions/h2o_submission_{y}_1.csv', index=False)